# Amazon CloudFront 컨텐츠 보호 2

## OAC = Origin Access Control
- OAI와 함께 클라우드프론트를 거치지 않은 S3 접근을 방지하는 기능
    - OAI = Origin Access Identity. 예전 방식. OAC를 권장
- OAC는 일종의 Identity : IAM 사용자 혹은 IAM 역할과 비슷한 Identity
    - 즉 S3에서 해당 OAC의 접근을 허용하고 클라우드프론트에서 OAC를 활용하여 S3와 소통
    - S3에서 기본적으로 모든 접근을 차단하고 OAC의 접근만 허용
- OAC는 Lambda Function URL에도 사용 가능

### Origin Access Control의 세가지 Sign 방법
- 클라우드프론트가 S3와 소통하기 위한 요청의 Sign 방법을 정의 가능
- 3가지 종류
    - Sigh Requests : CloudFront IAM Principle이 S3에 요청할 때 SigV4로 Sign
        - 즉 요청에 자격증명을 활용, 필요한 정보로 Authorization Header를 구성. S3에서 해당 내용을 검증하여 자격이 있는 요청인지 확인
        - 클라이언트가 Sign한 헤더가 있다면 덮어 씌우는 것이 특징
    - Do Not Override Authorization Header : 클라이언트 헤더가 있으면 그대로 사용, 없으면 새로 Sign
    - Do Not Requests : Authorization Header를 사용하지 않음
        - 클라이언트가 항상 Sign을 통해 요청 or 컨텐츠가 퍼블릭으로 공개된 경우

### 데모 - 클라우드프론트 OAC 설정
1. 버킷 생성
2. 클라우드프론트 배포 생성
    - 원본 S3 버킷 선택
    - 원본 엑세스 제어 설정
        - 새 OAC 생성
    - 캐시 레거시 세팅 = TTL 0
    - 보안 보호 비활성화
3. S3 버킷 정책 편집 = 정책 복사 붙여넣기
4. 버킷 정책을 삭제하면 어떻게 될까?
    - 바로 access denied

## Custom Origin 보호
- 방법 1: 커스텀 헤더 활용
    - 클라우드프론트에서 헤더 생성, 오리진에서 해당 헤더가 없으면 거부
- 방법 2: 오리진에서 클라우드프론트 IP를 제외한 모든 트래픽을 차단

### 데모 - 커스텀 오리진 보호
1. 인스턴스 생성
    - 키페어 없음
    - 보안그룹 디폴트
    - 유저데이터 스크립트 입력
2. 보안그룹 생성
    - 인바운드 규칙 추가
        - 모든 트래픽 from 접두사목록 = global.cloudfront.origin-facing
    - 인스턴스의 보안그룹 변경
3. 클라우드프론트 배포 생성
    - 오리진은 인스턴스 DNS
    - 프로토콜 HTTP만 해당
    - 레거시 캐시 세팅
        - 커스터마이징, 기본 TTL 0
    - 보안 보호 비활성화

### 지리적 배포 제한(Geo Restriction)
- 두 가지 접근 방법
    - 클라우드프론트 지리 배포 제한
        - Whitelist or Blacklist -> 나라별 기준
        - 모든 배포에 제한 사항 포함, 일부만 제한 걸기 불가능
        - IP주소의 정확도는 99.8%
    - 써드파티 지리적 위치 서비스 사용
        - 커스터마이징 가능(예: 브라우저 별로, 쿠키 별로 등등)
        - Signed URL 기반

### Field-Level Encryption
- 클라우드프론트를 활용하여 실제 데이터를 처리하는 주체까지 데잍를 암호화해서 전달할 수 있는 방법
- 엣지 로케이션에서 받은 데이터 중 특정 데이터를 주어진 퍼블릭키로 암호화
- 이후 데이터를 처리하는 측에서 프라이빗 키로 복호화하여 사용